## Dados AgroLink

`Dataset Amostral Cotações`

**Resumo**: Base de dados contendo as informações de COTAÇÕES disponibilizados pela AgroLink pós efeito de correções, e adaptações  <br>
**Fonte**: Dados internos

In [22]:
# Loading libraries
import pandas as pd
import numpy as np
import json
import requests
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import date

In [23]:
# Initial setting
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_colwidth = 100
pd.set_option('display.max_columns', None)

In [24]:
# URL Template
URL_TEMPLATE = {
    'STATE': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/tabelas/CotacoesFechamentoEstados',
    'CITIES': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/tabelas/CotacoesFechamentoCidades',
    'CITIES_FILTER': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/tabelas/CotacoesFechamentoCidades?uf={sigla_uf}&possuiCotacoes=false',
    'CROPS': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/tabelas/CotacoesFechamentoCulturas',
    'SPECIES': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/tabelas/GetEspecies',
    'PRODUCT': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/tabelas/CotacoesFechamentoProdutos',
    'PRODUCT_FILTER': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/tabelas/CotacoesFechamentoProdutos?cod_especie={cod_especie}',
#     'QUOTE_STATE': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/cotacoes/CotacoesFechamentoLista?uf={uf}',
#     'QUOTE_CITIES': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/cotacoes/CotacoesFechamentoLista?uf={uf}&cod_loc_ibge={cod_ibge}',
    'QUOTE_TOTAL': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/cotacoes/CotacoesFechamentoLista?data_inicial={data_inicial}&data_final={data_final}&limit={limit}&offset={offset}',
    'QUOTE_STATE': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/cotacoes/CotacoesFechamentoLista?uf={uf}&data_inicial={data_inicial}&data_final={data_final}&limit={limit}&offset={offset}',
    'QUOTE_CITIES': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/cotacoes/CotacoesFechamentoLista?uf={uf}&codCidadeIbge={cod_ibge}&cidade={cidade}&cod_especie={cod_especie}&cod_produto={cod_produto}&data_inicial={data_inicial}&data_final={data_final}&limit={limit}&offset={offset}',
    'AVG_QUOTE': 'https://api.agrolink.com.br/clientes/bradesco/968395037096845/cotacoes/CotacoesMediasLista?ano_ini={ano_ini}&mes_ini={mes_ini}&ano_fim={ano_fim}&mes_fim={mes_fim}&limit={limit}&offset={offset}'
}

In [25]:
def get_states():
    """Function to get the total UF list

    Return
    -------
    data: numpy.DataFrame
        Dataframe containing 2 distincts dimensions
        "codigo" and "descricao"    
    
    """
    
    # Setting parameters
    url = URL_TEMPLATE['STATE']

    # Getting the API information
    ## Alternative Running
    #json_file = requests.get(url).content
    json_file = requests.request('GET', url).content
    page = BeautifulSoup(json_file, 'html.parser', from_encoding="utf-8")

    # Building a dataframe based on the JSON output
    data = json.loads(page.text)
    data = pd.json_normalize(data)

    # Returning the result
    return(data)

In [26]:
def get_cities():
    """Function to get the total cities list of Brazilian States

    Return
    -------
    data: numpy.DataFrame
        Dataframe containing 5 distincts dimensions
        'cod_estado', 'uf', 'cod_loc', 'cod_loc_ibge' e 'Descricao'  
    
    """
    
    # Setting parameters
    url = URL_TEMPLATE['CITIES']

    # Getting the API information
    ## Alternative Running
    #json_file = requests.get(url).content
    json_file = requests.request('GET', url).content
    page = BeautifulSoup(json_file, 'html.parser', from_encoding="utf-8")

    # Building a dataframe based on the JSON output
    data = json.loads(page.text)
    data = pd.json_normalize(data)

    # Returning the result
    return(data)

In [27]:
def get_crops():
    """Function to get the entire list of Farm Crops

    Return
    -------
    data: numpy.DataFrame
        Dataframe containing 2 distincts dimensions
        "codigo" and "descricao"    
    
    """
    
    # Setting parameters
    url = URL_TEMPLATE['CROPS']

    # Getting the API information
    ## Alternative Running
    #json_file = requests.get(url).content
    json_file = requests.request('GET', url).content
    page = BeautifulSoup(json_file, 'html.parser', from_encoding="utf-8")

    # Building a dataframe based on the JSON output
    data = json.loads(page.text)
    data = pd.json_normalize(data)

    # Returning the result
    return(data)

In [28]:
def get_species():
    """Function to get the entire list of products

    Return
    -------
    data: numpy.DataFrame
        Dataframe containing 4 distincts dimensions
        "codEspecie", "nomeEspecie", "tipo" and "chave"
    
    """
    
    # Setting parameters
    url = URL_TEMPLATE['SPECIES']

    # Getting the API information
    ## Alternative Running
    #json_file = requests.get(url).content
    json_file = requests.request('GET', url).content
    page = BeautifulSoup(json_file, 'html.parser', from_encoding="utf-8")

    # Building a dataframe based on the JSON output
    data = json.loads(page.text)
    data = pd.json_normalize(data)

    # Exploding the created column, and normalizing it next
    data = data.explode(column = 'especies', ignore_index = True)
    data = pd.json_normalize(data = data['especies'])

    # Returning the result
    return(data)

In [29]:
def get_products():
    """Function to get the entire list of products

    Return
    -------
    data: numpy.DataFrame
        Dataframe containing 2 distincts dimensions
        "codigo" and "descricao"    
    
    """
    
    # Setting parameters
    url = URL_TEMPLATE['PRODUCT']

    # Getting the API information
    ## Alternative Running
    #json_file = requests.get(url).content
    json_file = requests.request('GET', url).content
    page = BeautifulSoup(json_file, 'html.parser', from_encoding="utf-8")

    # Building a dataframe based on the JSON output
    data = json.loads(page.text)
    data = pd.json_normalize(data)

    # Returning the result
    return(data)

In [30]:
# Collecting all UF codes
uf_list = get_states()
uf_list = uf_list['Descricao'].tolist()

In [31]:
def daily_total_quote(uf = uf_list, limit = 5000, data_inicial = '29/08/2022', data_final = date.today().strftime("%d/%m/%Y")):
    """Function to get the Daily Quote for all cultures and products by brazilian states
    
    Parameters
    -------    
    limit: integer
        limit of returns per page (max = 5000)
        
    data_inicial: date
        initial date in format of dd/mm/yyyy
        
    data_final: date
        final date in format of dd/mm/yyyy

    Return
    -------
    result: numpy.DataFrame
        Dataframe containing the quote information
        
    """
    
    # Setting initial parameters
    result = pd.DataFrame()
    limit = limit
    offset = 0
    start_time = time.time()
    
    # Starting message
    print('Starting Process!')
        
    # Running through all the predifined UF
    while True:

        # Setting parameters
        url = URL_TEMPLATE['QUOTE_TOTAL'].format(limit = limit, offset = offset,
                                                data_inicial = data_inicial, data_final = data_final)

        # Getting the API information
        ## Alternative Running
        #json_file = requests.get(url).content
        json_file = requests.request('GET', url).content
        page = BeautifulSoup(json_file, 'html.parser', from_encoding="utf-8")

        # Converting the result
        data = json.loads(page.text)

        # IF Condition
        if len(data) == 0:
            # Reset parameters, and exit the loop
            offset = 0
            end_time = time.time()
            print('Time Elapsed: ' + str(round(end_time - start_time, 2)) + ' seconds')
            print('Process Finished!')
            break

        # Building a dataframe based on the JSON output
        data = pd.json_normalize(data)
        total_row = data['TotalRows'][0]

        # Status of the iteration
        print('-----------------------------------')
        print('Completion Status ' + str(round(offset/total_row * 100, 1))+ '%')
        
        # Concatenating the multiple results
        result = pd.concat([result, data])
        
        # Running through all the pages
        offset = offset + limit
        
    # Returning the result
    return(result)

In [32]:
def daily_quote_by_state(uf = uf_list, limit = 5000, data_inicial = '29/08/2022', data_final = date.today().strftime("%d/%m/%Y")):
    """Function to get the Daily Quote for all cultures and products by brazilian states
    
    Parameters
    -------
    uf: list
        list containing the UF code
    
    limit: integer
        limit of returns per page (max = 5000)
        
    data_inicial: date
        initial date in format of dd/mm/yyyy
        
    data_final: date
        final date in format of dd/mm/yyyy

    Return
    -------
    result: numpy.DataFrame
        Dataframe containing the quote information
        
    """
    
    # Setting initial parameters
    result = pd.DataFrame()
    limit = limit
    offset = 0
    
     # Running through all the predifined UF
    for uf in tqdm(uf):
        
        while True:
            
            # Setting parameters
            url = URL_TEMPLATE['QUOTE_STATE'].format(uf = uf, limit = limit, offset = offset,
                                                    data_inicial = data_inicial, data_final = data_final)
            
            # Getting the API information
            ## Alternative Running
            #json_file = requests.get(url).content
            json_file = requests.request('GET', url).content
            page = BeautifulSoup(json_file, 'html.parser', from_encoding="utf-8")

            # Converting the result
            data = json.loads(page.text)
            
            # IF Condition
            if len(data) == 0:
                # Reset parameters, and exit the loop
                offset = 0
                break
            
            # Building a dataframe based on the JSON output
            data = pd.json_normalize(data)

            # Concatenating the multiple results
            result = pd.concat([result, data])
            
            # Running through all the pages
            offset = offset + limit
        
    # Returning the result
    return(result)

In [33]:
# # Getting the entire set of prices by Total
# df = daily_total_quote(limit = 1000)
# df = df.reset_index().drop('index', axis = 1)
# df.shape

In [23]:
# Getting the entire set of prices by State
df = daily_quote_by_state(limit = 1000, data_inicial= '04/12/2022', data_final= '06/12/2022')
df = df.reset_index().drop('index', axis = 1)
df.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:24<00:00,  1.11it/s]


,CodigoPrecoConteudo,cod_conteudo,CodigoPreco,cod_especie,produto_detalhe,Cultura,codigoproduto,NomeProduto,Classificacao,Embalagem,cod_grupo,nome_grupo,EmbalabemDescricao,Pagamento,Obs,Quantidade,Unidade,cod_estado,UF,cod_loc,cod_loc_ibge,Cidade,Data,Valor,UnidadeNegociacao,variacao,TotalRows
0,2384495,143,15807799,673,Açucar ATR 1kg/ton,Açúcar,4156,Açucar,,ATR,11,Diversos,None,A Vista,Média estadual,1.00,kg/ton,9823,AL,9941,2704302,Maceió,2022-12-06T00:00:00,1.15,None,0.00,4
1,2384523,143,15807798,670,Leite 1L,Leite,2161,Leite,,,11,Diversos,Litros,A Vista,A vista,1.00,L,9823,AL,9941,2704302,Maceió,2022-12-06T00:00:00,2.10,l,0.00,4
2,2379138,143,15803120,673,Açucar ATR 1kg/ton,Açúcar,4156,Açucar,,ATR,11,Diversos,None,A Vista,Média estadual,1.00,kg/ton,9823,AL,9941,2704302,Maceió,2022-12-05T00:00:00,1.15,None,0.00,4
3,2379166,143,15803119,670,Leite 1L,Leite,2161,Leite,,,11,Diversos,Litros,A Vista,A vista,1.00,L,9823,AL,9941,2704302,Maceió,2022-12-05T00:00:00,2.10,l,0.00,4
4,2383852,143,15812888,2,Milho Seco Sc 60Kg,Milho,1090,Milho Seco,,Sc,13,Grãos,Saca,A Vista,A vista,60.00,Kg,9826,BA,21278,2919553,Luís Eduardo Magalhães,2022-12-06T00:00:00,72.00,sc,0.00,685


In [15]:
# Exporting the final result
file_name = date.today().strftime("%Y%m%d")
df.to_excel('base_consolidada_agrolink_' + file_name + '.xlsx', index = False)

In [16]:
# Creating the year_month column
df['year_month'] = df['Data'].str[:4] + df['Data'].str[5:7]
df.head()

,CodigoPrecoConteudo,cod_conteudo,CodigoPreco,cod_especie,produto_detalhe,Cultura,codigoproduto,NomeProduto,Classificacao,Embalagem,cod_grupo,nome_grupo,EmbalabemDescricao,Pagamento,Obs,Quantidade,Unidade,cod_estado,UF,cod_loc,cod_loc_ibge,Cidade,Data,Valor,UnidadeNegociacao,variacao,TotalRows,year_month
0,629933,143,15614009,673,Açucar ATR 1kg/ton,Açúcar,4156,Açucar,,ATR,11,Diversos,None,A Vista,Média estadual,1.00,kg/ton,9823,AL,9941,2704302,Maceió,2022-10-04T00:00:00,1.28,None,0.00,54,202210
1,632549,143,15614008,670,Leite 1L,Leite,2161,Leite,,,11,Diversos,Litros,A Vista,A vista,1.00,L,9823,AL,9941,2704302,Maceió,2022-10-04T00:00:00,2.10,l,0.00,54,202210
2,627078,143,15609826,673,Açucar ATR 1kg/ton,Açúcar,4156,Açucar,,ATR,11,Diversos,None,A Vista,Média estadual,1.00,kg/ton,9823,AL,9941,2704302,Maceió,2022-10-03T00:00:00,1.28,None,0.00,54,202210
3,627105,143,15609825,670,Leite 1L,Leite,2161,Leite,,,11,Diversos,Litros,A Vista,A vista,1.00,L,9823,AL,9941,2704302,Maceió,2022-10-03T00:00:00,2.10,l,0.00,54,202210
4,619971,143,15605830,673,Açucar ATR 1kg/ton,Açúcar,4156,Açucar,,ATR,11,Diversos,None,A Vista,Média estadual,1.00,kg/ton,9823,AL,9941,2704302,Maceió,2022-09-30T00:00:00,1.28,None,0.00,54,202209


In [17]:
# Aggregating by UF
df_agg_uf = (df
            .groupby(['cod_especie', 
                        'Cultura', 
                        'codigoproduto', 
                        'produto_detalhe',
                        'NomeProduto', 
                        'Classificacao', 
                        'Embalagem', 
                        'Quantidade', 
                        'Unidade', 
                        'UF', 
                        'year_month'])
            .agg(media_UF = ('Valor', np.mean))
            .reset_index())
df_agg_uf.head()

,cod_especie,Cultura,codigoproduto,produto_detalhe,NomeProduto,Classificacao,Embalagem,Quantidade,Unidade,UF,year_month,media_UF
0,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,BA,202208,164.79
1,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,BA,202209,164.32
2,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,BA,202210,162.60
3,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,DF,202208,167.53
4,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,DF,202209,167.01


In [18]:
# Aggregating by BR
df_agg_br = (df
            .groupby(['cod_especie', 
                        'Cultura', 
                        'codigoproduto', 
                        'produto_detalhe',
                        'NomeProduto', 
                        'Classificacao', 
                        'Embalagem', 
                        'Quantidade', 
                        'Unidade', 
                        'year_month'])
            .agg(media_BR = ('Valor', np.mean))
            .reset_index())
df_agg_br.head()

,cod_especie,Cultura,codigoproduto,produto_detalhe,NomeProduto,Classificacao,Embalagem,Quantidade,Unidade,year_month,media_BR
0,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,202208,168.98
1,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,202209,169.26
2,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,202210,166.93
3,1,Soja,2181,Soja s/ Royalts Sc 60Kg,Soja s/ Royalts,,Sc,60.00,Kg,202208,171.33
4,1,Soja,2181,Soja s/ Royalts Sc 60Kg,Soja s/ Royalts,,Sc,60.00,Kg,202209,170.59


In [19]:
# Concatenating the final result
df_agg = pd.merge(df_agg_uf, 
                  df_agg_br[['cod_especie', 'codigoproduto', 'year_month', 'media_BR']],
                  how = 'left',
                  on = ['cod_especie', 'codigoproduto', 'year_month']
                 )
df_agg.head()

,cod_especie,Cultura,codigoproduto,produto_detalhe,NomeProduto,Classificacao,Embalagem,Quantidade,Unidade,UF,year_month,media_UF,media_BR
0,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,BA,202208,164.79,168.98
1,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,BA,202209,164.32,169.26
2,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,BA,202210,162.60,166.93
3,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,DF,202208,167.53,168.98
4,1,Soja,9,Soja em Grão Sc 60Kg,Soja,em Grão,Sc,60.00,Kg,DF,202209,167.01,169.26


In [20]:
# Exporting the consolidated result
file_name = date.today().strftime("%Y%m%d")
df_agg.to_excel('aggregated_base_agrolink_' + file_name + '.xlsx', index = False)

## Quotes - National and State Average

In [13]:
def get_national_and_state_avg_quote(start_year_month, final_year_month, limit = 5000):
    """Function to get the entire list of National and State Average Quotes
    
    Parameters
    -------
    start_year_month: integer
        full starting year_month (ex.: 202208)
    
    final_year_month: integer
        full final year_month (ex.: 202209)
        
    limit: integer
        limit of returns per page (max = 5000)

    Return
    -------
    result: numpy.DataFrame
        Dataframe containing the quote information
        
    """
    
    # Setting initial parameters
    result = pd.DataFrame()
    limit = limit
    offset = 0
    start_time = time.time()
    
    # Starting message
    print('Starting Process!')
    
    # Running through all the pages
    while True:

        # Setting parameters
        url = URL_TEMPLATE['AVG_QUOTE'].format(limit = limit, offset = offset,
                                               ano_ini = str(start_year_month)[:4], 
                                               mes_ini = str(start_year_month)[4:],
                                               ano_fim = str(final_year_month)[:4], 
                                               mes_fim = str(final_year_month)[4:])
        
        # Getting the API information
        ## Alternative Running
        #json_file = requests.get(url).content
        json_file = requests.request('GET', url).content
        page = BeautifulSoup(json_file, 'html.parser', from_encoding="utf-8")

        # Converting the result
        data = json.loads(page.text)

        # IF Condition
        if len(data) == 0:
            # Reset parameters, and exit the loop
            offset = 0
            end_time = time.time()
            print('Time Elapsed: ' + str(round(end_time - start_time, 2)) + ' seconds')
            print('Process Finished!')
            break

        # Building a dataframe based on the JSON output
        data = pd.json_normalize(data)
        total_row = data['TotalRows'][0]

        # Status of the iteration
        print('-----------------------------------')
        print('Completion Status ' + str(round(offset/total_row * 100, 1))+ '%')

        # Concatenating the multiple results
        result = pd.concat([result, data])

        # Running through all the pages
        offset = offset + limit
        
    # Returning the result
    return(result)

In [15]:
df_avg_quote = get_national_and_state_avg_quote(202208, 202211)
df_avg_quote.head()

Starting Process!
-----------------------------------
Completion Status 0.0%
Time Elapsed: 0.58 seconds
Process Finished!


,ano,mes,uf,codigoproduto,produto_detalhe,cultura,mediaEst,mediaNac,unidade,TotalRows
0,2022,11,BA,2140,Abacate Comum Cx 30Kg,Abacate,192.14,192.14,Cx 30Kg,1837
1,2022,11,PR,3842,Abacate Cx 20Kg,Abacate,69.50,69.50,Cx 20Kg,1837
2,2022,11,AL,4156,Açucar ATR 1kg/ton,Açúcar,1.15,1.15,ATR 1kg/ton,1837
3,2022,11,PE,4156,Açucar ATR 1kg/ton,Açúcar,1.31,1.15,ATR 1kg/ton,1837
4,2022,11,PR,4156,Açucar ATR 1kg/ton,Açúcar,1.00,1.15,ATR 1kg/ton,1837


In [23]:
# Exporting the consolidated result
file_name = date.today().strftime("%Y%m%d")
df_avg_quote.to_excel('average_quote_' + file_name + '.xlsx', index = False)